In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install neattext

     |████████████████████████████████| 114 kB 5.3 MB/s 


In [4]:
!pip3 install  pyLDAvis --quiet

     |████████████████████████████████| 1.7 MB 5.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 15.7 MB 41.8 MB/s 
     |████████████████████████████████| 11.3 MB 48.1 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.3.post1 requires numpy<1.20,>=1.16.0, but you have numpy 1.21.5 which is incompatible.
google-colab 1.0.0 requires pandas~=1.1.0; python_version >= "3.0", but you have pandas 1.3.5 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [5]:
import pandas as pd
import numpy as np
from pprint import pprint

import matplotlib.pyplot as plt

# spacy for lemmatization
import spacy

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel


import plotly.express as px
import nltk
nltk.download('all')
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import neattext.functions as nf
from textblob import TextBlob, Word
from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import CountVectorizer
import os.path
from gensim import corpora
from gensim.models import LsiModel
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import display
from tqdm import tqdm
from collections import Counter
import ast

import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import seaborn as sb

from sklearn.feature_extraction.text import CountVectorizer
from textblob import TextBlob
import scipy.stats as stats

from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.manifold import TSNE

from bokeh.plotting import figure, output_file, show
from bokeh.models import Label
from bokeh.io import output_notebook
output_notebook()

%matplotlib inline

from bokeh.plotting import figure, output_file, show
from bokeh.models import Label
from bokeh.io import output_notebook
output_notebook()

%matplotlib inline
import re
import spacy
import string

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis # don't skip this

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipp

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning:

Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working



In [6]:
path = '/content/drive/MyDrive/T5camp/Anime_project/'

anime = pd.read_csv(path + 'animes.csv')
# profiles = pd.read_csv(path + 'profiles.csv')
# reviews = pd.read_csv(path + 'reviews.csv')

In [7]:
pd.options.display.max_colwidth = 500
syn = anime['synopsis']
syn

0        Following their participation at the Inter-High, the Karasuno High School volleyball team attempts to refocus their efforts, aiming to conquer the Spring tournament instead.  \r\n \r\nWhen they receive an invitation from long-standing rival Nekoma High, Karasuno agrees to take part in a large training camp alongside many notable volleyball teams in Tokyo and even some national level players. By playing with some of the toughest teams in Japan, they hope not only to sharpen their skills, but ...
1        Music accompanies the path of the human metronome, the prodigious pianist Kousei Arima. But after the passing of his mother, Saki Arima, Kousei falls into a downward spiral, rendering him unable to hear the sound of his own piano. \r\n  \r\nTwo years later, Kousei still avoids the piano, leaving behind his admirers and rivals, and lives a colorless life alongside his friends Tsubaki Sawabe and Ryouta Watari. However, everything changes when he meets a beautiful violinist, Kaori

In [8]:
syn = syn.astype('str')
syn = syn.apply(lambda text: text.lower())
syn = syn.apply(nf.remove_multiple_spaces)
syn = syn.apply(nf.remove_puncts)
syn = syn.apply(nf.remove_stopwords)
syn = syn.apply(nf.remove_numbers)
syn = syn.apply(nf.remove_special_characters)

In [9]:
# make counter for words
from collections import Counter
cnt = Counter()
for text in syn.values:
    for word in text.split():
        cnt[word] += 1
        
cnt.most_common(10)

[('source', 7131),
 ('world', 3530),
 ('school', 3381),
 ('new', 3153),
 ('mal', 2656),
 ('life', 2648),
 ('ann', 2530),
 ('written', 2503),
 ('rewrite', 2414),
 ('girl', 2395)]

In [10]:
#remove rare words
n_rare_words = 10
RAREWORDS = set([w for (w, wc) in cnt.most_common()[:-n_rare_words-1:-1]])
def remove_rarewords(text):
    """custom function to remove the rare words"""
    return " ".join([word for word in str(text).split() if word not in RAREWORDS])
syn = syn.apply(lambda text: remove_rarewords(text))
syn

0        following participation interhigh karasuno high school volleyball team attempts refocus efforts aiming conquer spring tournament instead receive invitation longstanding rival nekoma high karasuno agrees large training camp alongside notable volleyball teams tokyo national level players playing toughest teams japan hope sharpen skills come new attacks strengthen hinata kageyama attempt devise powerful weapon possibly break sturdiest blocks facing chance victory senior players graduate members...
1        music accompanies path human metronome prodigious pianist kousei arima passing mother saki arima kousei falls downward spiral rendering unable hear sound piano years later kousei avoids piano leaving admirers rivals lives colorless life alongside friends tsubaki sawabe ryouta watari changes meets beautiful violinist kaori miyazono stirs world sets journey face music based manga series shigatsu wa kimi uso approaches story kouseis recovery discovers music playing note perfectly 

In [11]:
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word,'v') for word in text.split()])

syn = syn.apply(lambda text: lemmatize_words(text))
syn

0        follow participation interhigh karasuno high school volleyball team attempt refocus efforts aim conquer spring tournament instead receive invitation longstanding rival nekoma high karasuno agree large train camp alongside notable volleyball team tokyo national level players play toughest team japan hope sharpen skills come new attack strengthen hinata kageyama attempt devise powerful weapon possibly break sturdiest block face chance victory senior players graduate members karasunos volleybal...
1        music accompany path human metronome prodigious pianist kousei arima pass mother saki arima kousei fall downward spiral render unable hear sound piano years later kousei avoid piano leave admirers rival live colorless life alongside friends tsubaki sawabe ryouta watari change meet beautiful violinist kaori miyazono stir world set journey face music base manga series shigatsu wa kimi uso approach story kouseis recovery discover music play note perfectly single melody bring fresh

In [12]:
# !pip install pyspellchecker
# spell = SpellChecker()
# for word in syn:
#   print(spell.unknown(word.split()))
#####################

#   from spellchecker import SpellChecker

# spell = SpellChecker()

# def correct_spellings(text):
#     corrected_text = []
#     misspelled_words = spell.unknown(text.split())
#     for word in text.split():
#         if word in misspelled_words:
#             corrected_text.append(spell.correction(word))
#         else:
#             corrected_text.append(word)
#     return " ".join(corrected_text)
        
# correct_spellings(syn)
# syn

In [13]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

syn = list(sent_to_words(syn))

In [14]:
# Create Dictionary
id2word = corpora.Dictionary(syn)

# Create Corpus
texts = syn

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [15]:
id2word[76]

'accompany'

In [16]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('agree', 1),
  ('aim', 1),
  ('alongside', 1),
  ('aoba', 1),
  ('archrival', 1),
  ('attack', 1),
  ('attempt', 2),
  ('block', 1),
  ('break', 1),
  ('camp', 1),
  ('chance', 1),
  ('come', 1),
  ('conquer', 1),
  ('devise', 1),
  ('differences', 1),
  ('efforts', 1),
  ('face', 1),
  ('follow', 1),
  ('formidable', 1),
  ('graduate', 1),
  ('harder', 1),
  ('high', 2),
  ('hinata', 1),
  ('hope', 2),
  ('instead', 1),
  ('interhigh', 1),
  ('invitation', 1),
  ('japan', 1),
  ('jousai', 1),
  ('kageyama', 1),
  ('karasuno', 2),
  ('karasunos', 1),
  ('large', 1),
  ('learn', 1),
  ('level', 1),
  ('longstanding', 1),
  ('mal', 1),
  ('members', 1),
  ('national', 1),
  ('nekoma', 1),
  ('new', 1),
  ('newincluding', 1),
  ('notable', 1),
  ('oikawa', 1),
  ('old', 1),
  ('opponents', 1),
  ('overcome', 1),
  ('participation', 1),
  ('play', 1),
  ('players', 2),
  ('possibly', 1),
  ('powerful', 1),
  ('receive', 1),
  ('refocus', 1),
  ('rewrite', 1),
  ('rival', 1),
  ('school'

In [ ]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin in

In [ ]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=syn, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word)
vis